Install Require Libraries

In [1]:
!pip install pandas numpy

Import Libraries

In [2]:
import pandas as pd
import numpy as np

# Show all columns in the dataframe
pd.set_option('display.max_columns', None)

Read CSV File

In [3]:
# Read File
leads_data = pd.read_csv('../data/Michigan.csv', on_bad_lines='skip')



/tmp/ipykernel_91441/1174662041.py:2: DtypeWarning: Columns (12,22,53,56,58) have mixed types. Specify dtype option on import or set low_memory=False.
  leads_data = pd.read_csv('../data/Michigan.csv', on_bad_lines='skip')


In [4]:
# Charge first 100 records of a template reference
template = pd.read_csv('../data/LIST_100_20240723-161841.csv', nrows=100)

In [5]:
template.columns

Index(['lead_id', 'entry_date', 'modify_date', 'status', 'status_name',
       'status_category_id', 'status_category_name', 'user',
       'vendor_lead_code', 'source_id', 'list_id', 'gmt_offset_now',
       'called_since_last_reset', 'phone_code', 'phone_number', 'title',
       'first_name', 'middle_initial', 'last_name', 'address1', 'address2',
       'address3', 'city', 'state', 'province', 'postal_code', 'country_code',
       'gender', 'date_of_birth', 'alt_phone', 'email', 'security_phrase',
       'comments', 'called_count', 'last_local_call_time', 'rank', 'owner',
       'entry_list_id'],
      dtype='object')

In [6]:
template.sample(2)

,lead_id,entry_date,modify_date,status,status_name,status_category_id,status_category_name,user,vendor_lead_code,source_id,list_id,gmt_offset_now,called_since_last_reset,phone_code,phone_number,title,first_name,middle_initial,last_name,address1,address2,address3,city,state,province,postal_code,country_code,gender,date_of_birth,alt_phone,email,security_phrase,comments,called_count,last_local_call_time,rank,owner,entry_list_id
80,1256874,2024-05-28 22:59:59,0000-00-00 00:00:00,NaN,No Answer AutoDial,UNDEFINED,Default Category -,NaN,NaN,NaN,100,-4.0,N,1,8024855455,OWNE,SCOTT WRIGHT,NaN,SCOTT WRIGHT,90 PLEASANT ST,NaN,NaN,WASHINGTON,VT,NaN,5663,NaN,U,0000-00-00,NaN,NaN,NaN,NaN,4,2008-01-01 00:00:00,0,NaN,0
47,1256841,2024-05-28 22:59:59,0000-00-00 00:00:00,DNC,DO NOT CALL,UNDEFINED,Default Category -,NaN,NaN,NaN,100,-4.0,N,1,3023799025,ASSI,MS SOFIA CRUZ,NaN,MORGAN COSMETICS LLC,2509 248TH PL NE,NaN,NaN,KING,WA,NaN,98074,NaN,U,0000-00-00,NaN,NaN,NaN,NaN,3,2008-01-01 00:00:00,0,NaN,0


In [7]:
leads_data.columns

Index(['Full name', 'Industry', 'Job title', 'Sub Role', 'Industry 2',
       'Emails', 'Mobile', 'Phone numbers', 'Company Name', 'Company Industry',
       'Company Website', 'Company Size', 'Company Founded', 'Location',
       'Locality', 'Metro', 'Region', 'Skills', 'First Name', 'Middle Initial',
       'Middle Name', 'Last Name', 'Birth Year', 'Birth Date', 'Gender',
       'LinkedIn Url', 'LinkedIn Username', 'Facebook Url',
       'Facebook Username', 'Twitter Url', 'Twitter Username', 'Github Url',
       'Github Username', 'Company Linkedin Url', 'Company Facebook Url',
       'Company Twitter Url', 'Company Location Name',
       'Company Location Locality', 'Company Location Metro',
       'Company Location Region', 'Company Location Geo',
       'Company Location Street Address', 'Company Location Address Line 2',
       'Company Location Postal Code', 'Company Location Country',
       'Company Location Continent', 'Last Updated', 'Start Date',
       'Job Summary', 'Loc

In [8]:
len(leads_data)

2127118

In [9]:
# Obtener el conteo de valores nulos por columna
null_counts = leads_data.isnull().sum()

# Convertir a DataFrame
null_counts_df = null_counts.reset_index()
null_counts_df.columns = ['Column', 'Null Count']

# Exportar a CSV
null_counts_df.to_csv('null_counts.csv', index=False)

Delete the rows without a phone number

In [10]:
df_cleaned = leads_data.dropna(subset=['Mobile', 'Phone numbers'], how='all')


Format data for the final status

In [11]:
# Function to process phone numbers
def process_phone_numbers(df):
    # Initialize new columns using .loc to avoid SettingWithCopyWarning
    df.loc[:, 'phone_code'] = np.nan
    df.loc[:, 'phone_number'] = np.nan
    df.loc[:, 'alt_phone'] = np.nan
    
    # Convert columns to string to avoid attribute errors
    df['Mobile'] = df['Mobile'].astype(str)
    df['Phone numbers'] = df['Phone numbers'].astype(str)

    def extract_phone_info(phone):
        # Handle missing values
        if phone.lower() == 'nan':
            return (np.nan, np.nan)
        
        phone = phone.replace(' ', '')
        
        if phone.startswith('+'):
            phone = phone[1:]
            if len(phone) > 10 and len(phone) <= 12:
                phone_code = phone[:-10]
                phone_number = phone[-10:]
                return (phone_code, phone_number)
        return (np.nan, np.nan)

    def fill_phone_info(row):
        # Process Mobile column
        if row['Mobile'].lower() != 'nan':
            phone_code, phone_number = extract_phone_info(row['Mobile'])
            row['phone_code'] = phone_code
            row['phone_number'] = phone_number
        
        # Process Phone numbers column if Mobile is not valid
        if pd.isna(row['phone_code']) or pd.isna(row['phone_number']):
            phones = row['Phone numbers'].split(',')
            phone_code, phone_number = extract_phone_info(phones[0])
            row['phone_code'] = phone_code
            row['phone_number'] = phone_number
            
            # If there are additional numbers, add them to alt_phone
            if len(phones) > 1:
                row['alt_phone'] = ','.join(phones[1:])
        
        # If Mobile had valid data, just put Phone numbers in alt_phone
        else:
            row['alt_phone'] = row['Phone numbers']
        
        return row
    
    # Apply the function to each row of the DataFrame
    df = df.apply(fill_phone_info, axis=1)
    
    return df


# Procesar los números de teléfono
df_cleaned = process_phone_numbers(df_cleaned)

# Mostrar el DataFrame resultante
df_cleaned


/tmp/ipykernel_91441/1085458843.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'phone_code'] = np.nan
/tmp/ipykernel_91441/1085458843.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'phone_number'] = np.nan
/tmp/ipykernel_91441/1085458843.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_g

,Full name,Industry,Job title,Sub Role,Industry 2,Emails,Mobile,Phone numbers,Company Name,Company Industry,Company Website,Company Size,Company Founded,Location,Locality,Metro,Region,Skills,First Name,Middle Initial,Middle Name,Last Name,Birth Year,Birth Date,Gender,LinkedIn Url,LinkedIn Username,Facebook Url,Facebook Username,Twitter Url,Twitter Username,Github Url,Github Username,Company Linkedin Url,Company Facebook Url,Company Twitter Url,Company Location Name,Company Location Locality,Company Location Metro,Company Location Region,Company Location Geo,Company Location Street Address,Company Location Address Line 2,Company Location Postal Code,Company Location Country,Company Location Continent,Last Updated,Start Date,Job Summary,Location Country,Location Continent,Street Address,Address Line 2,Postal Code,Location Geo,Last Updated.1,Linkedin Connections,Inferred Salary,Years Experience,Summary,Countries,Interests,phone_code,phone_number,alt_phone
4,nicholas stauder,information technology and services,hpc systems administrator,NaN,NaN,"nstauder@gmail.com, nstauder@ix.netcom.com, ns...",+17344746913,"+17344746913, +13134341535",university of michigan,higher education,umich.edu,10001+,1817,"ann arbor, michigan, united states",ann arbor,"ann arbor, michigan",michigan,"architectural design, cad, microsoft office, p...",nicholas,j,john,stauder,1988.0,1988-07-05,male,linkedin.com/in/nicholas-stauder-a83b2822,nicholas-stauder-a83b2822,facebook.com/nstauder,nstauder,twitter.com/nstauder,nstauder,github.com/nstauder,nstauder,linkedin.com/company/university-of-michigan,NaN,NaN,"ann arbor, michigan, united states",ann arbor,"ann arbor, michigan",michigan,"42.27,-83.72",1239 kipke drive,NaN,48109,united states,north america,2020-07-01,2018-08,* High Performance Computing (HPC) systems adm...,united states,north america,300 briarcrest drive,unit 147,48104,"42.27,-83.72",2020-09-01,86,"150,000-250,000",10,Experienced systems administrator with a demon...,united states,"football, children, computers, design, technol...",1,7344746913,"+17344746913, +13134341535"
5,nicholas stauder,information technology and services,hpc systems administrator,NaN,NaN,"nstauder@gmail.com, nstauder@ix.netcom.com, ns...",+17344746913,"+17344746913, +13134341535",university of michigan,higher education,umich.edu,10001+,1817,"ann arbor, michigan, united states",ann arbor,"ann arbor, michigan",michigan,"architectural design, cad, microsoft office, p...",nicholas,j,john,stauder,1988.0,1988-07-05,male,linkedin.com/in/nicholas-stauder-a83b2822,nicholas-stauder-a83b2822,facebook.com/nstauder,nstauder,twitter.com/nstauder,nstauder,github.com/nstauder,nstauder,linkedin.com/company/university-of-michigan,NaN,NaN,"ann arbor, michigan, united states",ann arbor,"ann arbor, michigan",michigan,"42.27,-83.72",1239 kipke drive,NaN,48109,united states,north america,2020-07-01,2018-08,* High Performance Computing (HPC) systems adm...,united states,north america,300 briarcrest drive,unit 147,48104,"42.27,-83.72",2020-09-01,86,"150,000-250,000",10,Experienced systems administrator with a demon...,united states,"football, children, computers, design, technol...",1,7344746913,"+17344746913, +13134341535"
20,judy stull,retail,sales,accounts,sales,"hjstull49@gmail.com, riley.stull@gmail.com",+17022038753,"+17022038753, +17028385224, +18053790755, +161...",bath & body works,retail,bathandbodyworks.com,10001+,1990,"kalamazoo, michigan, united states",kalamazoo,"kalamazoo, michigan",michigan,NaN,judy,r,rae,stull,1949.0,NaN,female,linkedin.com/in/judy-stull-208b4644,judy-stull-208b4644,facebook.com/judy.stull,judy.stull,NaN,NaN,NaN,NaN,linkedin.com/company/bath-&-body-works,facebook.com/bathandbodyworks,twitter.com/luvbbw,"columbus, ohio, united states",columbus,"columbus, ohio",ohio,"39.96,-82.99",7 limited parkway,NaN,43068,united states,north america,2020-12-01,NaN,NaN,united states,north america,6209 poppy street,NaN,49009,"42.29,-85.58",2020-10-01,26,"70,000-85,000",3,"sales at Wil

Rename the name of some columns

In [12]:
df_cleaned.sample(2)

,Full name,Industry,Job title,Sub Role,Industry 2,Emails,Mobile,Phone numbers,Company Name,Company Industry,Company Website,Company Size,Company Founded,Location,Locality,Metro,Region,Skills,First Name,Middle Initial,Middle Name,Last Name,Birth Year,Birth Date,Gender,LinkedIn Url,LinkedIn Username,Facebook Url,Facebook Username,Twitter Url,Twitter Username,Github Url,Github Username,Company Linkedin Url,Company Facebook Url,Company Twitter Url,Company Location Name,Company Location Locality,Company Location Metro,Company Location Region,Company Location Geo,Company Location Street Address,Company Location Address Line 2,Company Location Postal Code,Company Location Country,Company Location Continent,Last Updated,Start Date,Job Summary,Location Country,Location Continent,Street Address,Address Line 2,Postal Code,Location Geo,Last Updated.1,Linkedin Connections,Inferred Salary,Years Experience,Summary,Countries,Interests,phone_code,phone_number,alt_phone
1589692,jason judkins,automotive,program manager,NaN,operations,bjudkins@latinmail.com,+15864773496,"+15864773496, +15867816786, +15869929720",conform automotive,NaN,NaN,NaN,NaN,"leonard, michigan, united states",leonard,"detroit, michigan",michigan,"engineering, manufacturing, automotive, autoca...",jason,w,william,judkins,1985.0,1985,male,linkedin.com/in/jason-judkins-8918688a,jason-judkins-8918688a,facebook.com/jason.judkins.5,jason.judkins.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-11-01,2014-12-01,NaN,united states,north america,2601 kingston road,NaN,48367.0,"42.86,-83.14",2020-10-01,145.0,"100,000-150,000",14.0,NaN,united states,"collecting antiques, exercise, sweepstakes, ho...",1,5864773496,"+15864773496, +15867816786, +15869929720"
1335454,cindy cloninger,primary/secondary education,secretary,NaN,NaN,NaN,nan,+17065669465,early college academy of columbus,NaN,NaN,NaN,NaN,"ithaca, michigan, united states",ithaca,NaN,michigan,NaN,cindy,NaN,NaN,cloninger,NaN,NaN,female,linkedin.com/in/cindy-cloninger-625a0077,cindy-cloninger-625a0077,facebook.com/cindy.cloninger.33,cindy.cloninger.33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-10-01,NaN,NaN,united states,north america,2540 south alger road,NaN,48847,"43.29,-84.60",2020-10-01,5,NaN,NaN,Secretary at Early College Academy of Columbus,united states,NaN,1,7065669465,NaN


Procesing the name into columns

In [13]:
# Columns to rename and their new names
columns_to_rename = {
    "Full name": "first_name",
    "Company Name": "last_name",
    "Street Address" : "address1",
    "Address Line 2" : "address2",
    "Location" : "city",
    "Postal Code": "postal_code",
    "Birth Date": "date_of_birth",
    "Emails" : "email",
    "Job title" : "title",
    "Gender" : "gender",
    "Middle Initial" : "middle_initial"
}

df_cleaned.rename(columns=columns_to_rename, inplace=True)

# Add a column for the state
df_cleaned['state'] = 'MI'

Upper case for all values

In [14]:
# Función para convertir caracteres no numéricos a mayúsculas
def convert_non_numeric_to_upper(s):
    if pd.isna(s):
        return s
    return ''.join([char.upper() if not char.isdigit() else char for char in s])

# Aplicar la función a cada columna del DataFrame
df_cleaned = df_cleaned.applymap(lambda x: convert_non_numeric_to_upper(str(x)))

/tmp/ipykernel_91441/573958443.py:8: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_cleaned = df_cleaned.applymap(lambda x: convert_non_numeric_to_upper(str(x)))


In [15]:
df_cleaned.sample(5)

,first_name,Industry,title,Sub Role,Industry 2,email,Mobile,Phone numbers,last_name,Company Industry,Company Website,Company Size,Company Founded,city,Locality,Metro,Region,Skills,First Name,middle_initial,Middle Name,Last Name,Birth Year,date_of_birth,gender,LinkedIn Url,LinkedIn Username,Facebook Url,Facebook Username,Twitter Url,Twitter Username,Github Url,Github Username,Company Linkedin Url,Company Facebook Url,Company Twitter Url,Company Location Name,Company Location Locality,Company Location Metro,Company Location Region,Company Location Geo,Company Location Street Address,Company Location Address Line 2,Company Location Postal Code,Company Location Country,Company Location Continent,Last Updated,Start Date,Job Summary,Location Country,Location Continent,address1,address2,postal_code,Location Geo,Last Updated.1,Linkedin Connections,Inferred Salary,Years Experience,Summary,Countries,Interests,phone_code,phone_number,alt_phone,state
1573013,ED FIZELL,AUTOMOTIVE,DIRECTOR ENGINEERING SERVICES,NAN,ENGINEERING,"EDFIZELL@YAHOO.COM, ED.FIZELL@GMAIL.COM, ED.FI...",NAN,+19524451680,DELPHI,AUTOMOTIVE,DELPHI.COM,10001+,2009,"FLINT, MICHIGAN, UNITED STATES",FLINT,"FLINT, MICHIGAN",MICHIGAN,"APQP, MANUFACTURING ENGINEERING, MANUFACTURING...",ED,NAN,NAN,FIZELL,NAN,NAN,MALE,LINKEDIN.COM/IN/ED-FIZELL-66325712,ED-FIZELL-66325712,NAN,NAN,NAN,NAN,NAN,NAN,LINKEDIN.COM/COMPANY/DELPHI,FACEBOOK.COM/DELPHI,TWITTER.COM/DELPHIAUTO,"KENT, KENT, UNITED KINGDOM",KENT,NAN,KENT,NAN,NAN,NAN,NAN,UNITED KINGDOM,EUROPE,2020-11-01,2019-04,NAN,UNITED STATES,NORTH AMERICA,NAN,NAN,NAN,"43.01,-83.68",2020-10-01,443.0,">250,000",37,CHIEF MANUFACTURING ENGINEER WITH 31+ YEARS OF...,UNITED STATES,NAN,1,9524451680,NAN,MI
206281,DAVID ASTRAUCKAS,AUTOMOTIVE,DISTRICT AND ACCOUNT MANAGER - GENERAL MOTORS,ACCOUNTS,SALES,"DAVID.ASTRAUCKAS@FANUCROBOTICS.COM, DAVID.ASTR...",NAN,+12482768403,FANUC AMERICA CORPORATION,INDUSTRIAL AUTOMATION,FANUCAMERICA.COM,501-1000,1982,"DETROIT, MICHIGAN, UNITED STATES",DETROIT,"DETROIT, MICHIGAN",MICHIGAN,NAN,DAVID,NAN,NAN,ASTRAUCKAS,NAN,NAN,MALE,LINKEDIN.COM/IN/DAVID-ASTRAUCKAS-37715611,DAVID-ASTRAUCKAS-37715611,NAN,NAN,NAN,NAN,NAN,NAN,LINKEDIN.COM/COMPANY/FANUC-AMERICA-CORPORATION,FACEBOOK.COM/FANUC,TWITTER.COM/FANUCAMERICA,"ROCHESTER HILLS, MICHIGAN, UNITED STATES",ROCHESTER HILLS,"DETROIT, MICHIGAN",MICHIGAN,"42.65,-83.15",3900 WEST HAMLIN,NAN,48309,UNITED STATES,NORTH AMERICA,2020-12-01,2015,NAN,UNITED STATES,NORTH AMERICA,NAN,NAN,NAN,"42.33,-83.04",2020-09-01,448,"150,000-250,000",22,FANUC AMERICA - DIRECTOR SALES,UNITED STATES,NAN,1,2482768403,NAN,MI
1478739,CODY HARDIN,TRANSPORTATION/TRUCKING/RAILROAD,NAN,NAN,NAN,CHARDIN@BELLETIRE.COM,NAN,+15867594343,NAN,NAN,NAN,NAN,NAN,"ALLEN PARK, MICHIGAN, UNITED STATES",ALLEN PARK,"DETROIT, MICHIGAN",MICHIGAN,"INVENTORY MANAGEMENT, TRAINING, MANAGEMENT, CU...",CODY,NAN,NAN,HARDIN,NAN,NAN,MALE,LINKEDIN.COM/IN/CODY-HARDIN-00712281,CODY-HARDIN-00712281,NAN,NAN,NAN,NAN,NAN,NAN,NAN,NAN,NAN,NAN,NAN,NAN,NAN,NAN,NAN,NAN,NAN,NAN,NAN,NAN,NAN,NAN,UNITED STATES,NORTH AMERICA,NAN,NAN,NAN,"42.25,-83.21",2020-10-01,2.0,NAN,19.0,NAN,UNITED STATES,NAN,1,5867594343,NAN,MI
1218660,DOUG BENTLEY,INFORMATION TECHNOLOGY AND SERVICES,NAN,NAN,NAN,DOUG.BENTLEY@TRUVENHEALTH.COM,NAN,+17343899682,NAN,NAN,NAN,NAN,NAN,"SALINE, MICHIGAN, UNITED STATES",SALINE,"ANN ARBOR, MICHIGAN",MICHIGAN,"WORDSMITHING, INTERNAL COMMUNICATIONS, CRISIS ...",DOUG,NAN,NAN,BENTLEY,NAN,NAN,MALE,LINKEDIN.COM/IN/DOUG-BENTLEY-37351783,DOUG-BENTLEY-37351783,FACEBOOK.COM/DOUG.BENTLEY,DOUG.BENTLEY,NAN,NAN,NAN,NAN,NAN,NAN,NAN,NAN,NAN,NAN,NAN,NAN,NAN,NAN,NAN,NAN,NAN,NAN,NAN,NAN,UNITED STATES,NORTH AMERICA,NAN,NAN,NAN,"42.16,-83.78",2020-11-01,285,"45,000-55,000",27.0,INTERNAL COMMUNICATIONS AT TRUVEN HEALTH ANALY...,UNITED STATES,NAN,1,7343899682,NAN,MI
1376042,ALAN MARTIN,LOGISTICS AND SUPPLY CHAIN,"EXPERIENCED EXECUTIVE, FINANCE, IT, HUMAN RESO...",NAN,FINANCE,AMARTIN@MAGLINER.COM,NAN,"+19898792411, +19895121000",MARTIN ENTERPRISES,NAN,MARTINFRP.COM

Delete extra columns

In [16]:
df_cleaned.columns

Index(['first_name', 'Industry', 'title', 'Sub Role', 'Industry 2', 'email',
       'Mobile', 'Phone numbers', 'last_name', 'Company Industry',
       'Company Website', 'Company Size', 'Company Founded', 'city',
       'Locality', 'Metro', 'Region', 'Skills', 'First Name', 'middle_initial',
       'Middle Name', 'Last Name', 'Birth Year', 'date_of_birth', 'gender',
       'LinkedIn Url', 'LinkedIn Username', 'Facebook Url',
       'Facebook Username', 'Twitter Url', 'Twitter Username', 'Github Url',
       'Github Username', 'Company Linkedin Url', 'Company Facebook Url',
       'Company Twitter Url', 'Company Location Name',
       'Company Location Locality', 'Company Location Metro',
       'Company Location Region', 'Company Location Geo',
       'Company Location Street Address', 'Company Location Address Line 2',
       'Company Location Postal Code', 'Company Location Country',
       'Company Location Continent', 'Last Updated', 'Start Date',
       'Job Summary', 'Location Co

In [17]:
# Seleccionar solo las columnas deseadas
columns_to_keep = ['first_name', 'middle_initial', 'last_name', 'title', 'email',
                    'phone_code', 'phone_number', 'alt_phone', 'address1', 'address2',
                    'city', 'state', 'postal_code', 'date_of_birth', ]
df_cleaned = df_cleaned[columns_to_keep]

In [18]:
df_cleaned.sample(5)

,first_name,middle_initial,last_name,title,email,phone_code,phone_number,alt_phone,address1,address2,city,state,postal_code,date_of_birth
1001232,DENIS REICHARD,NAN,COLLINS & ASSOCIATES,ACCOUNT EXECUTIVE,"DJAYREICHARD@YAHOO.COM, DENIS@IEUTER.COM",1,9898356701,NAN,NAN,NAN,"BAY CITY, MICHIGAN, UNITED STATES",MI,NAN,NAN
1945836,BARTT BRENTON,NAN,BRENTON USA,PRESIDENT,BRENTONB@MSU.EDU,1,6166087424,NAN,NAN,NAN,"LANSING, MICHIGAN, UNITED STATES",MI,NAN,NAN
226207,ROGER BRYNER,F,DJS SYSTEMS INC,PROJECT ENGINEER,"ROGER.BRYNER@GMAIL.COM, CHUCKSANDBERG1@AOL.COM...",1,2695807934,"+12695807934, +12697633676",NAN,NAN,"KALAMAZOO, MICHIGAN, UNITED STATES",MI,NAN,1961-05-04
1372107,ANTHONY LAUER,NAN,ANN ARBOR PUBLIC SCHOOLS,OPTIONS AND ONLINE PROGRAM COORDINATOR FOR ANN...,"AALAUER@YAHOO.COM, AALAUER@HOTMAIL.COM, LAUERA...",1,7349942330,"+17344879203, +17344766863",2533 FOSTER AVENUE,NAN,"ANN ARBOR, MICHIGAN, UNITED STATES",MI,48108,NAN
1552109,ANGELA SHING,NAN,GM FINANCIAL,SENIOR SYSTEM ANALYST - SALESFORCE.COM AT GENE...,"ANGELA.SHING@DIALOG-DIRECT.COM, ANGIESHING@HOT...",1,3139575100,NAN,NAN,NAN,"STERLING HEIGHTS, MICHIGAN, UNITED STATES",MI,NAN,NAN
